In [4]:
##imports
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from keras.models import load_model
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os

### Description

The image-set, which is contained in the **validation_set** folder is a dedicated data-set with images captured in several parts of the day with various lighting. This dataset is different to the one used for the training of the models and aims to measure the accuracy of a model in images it has not seen before. Example of a few images can be shown here:

<table>
  <tr>
    <td><img src="validation_set/A111.jpg" width=100 height=100></td>
    <td><img src="validation_set/A3300.jpg" width=100 height=100></td>
    <td><img src="validation_set/B3144.jpg" width=100 height=100></td>
    <td><img src="validation_set/B102.jpg" width=100 height=100></td>
  </tr>
   <tr>
    <td><img src="validation_set/B186.jpg" width=100 height=100></td>
    <td><img src="validation_set/U111.jpg" width=100 height=100></td>
    <td><img src="validation_set/U3072.jpg" width=100 height=100></td>
    <td><img src="validation_set/V66.jpg" width=100 height=100></td>
  </tr>
   <tr>
    <td><img src="validation_set/V66.jpg" width=100 height=100></td>
    <td><img src="validation_set/E21.jpg" width=100 height=100></td>
    <td><img src="validation_set/EE33.jpg" width=100 height=100></td>
    <td><img src="validation_set/A228.jpg" width=100 height=100></td>
  </tr>
</table>
 
 This validation set contains 5 Classes:
 
 - Class 0 = fist
 - Class 1 = palm
 - Class 2 = index & middle finger
 - Class 3 = V
 - Class 4 = empty

### Load-Set

In this section we are gonna load the validation set and save the raw images, as well as the images after applying to them image processing in order to examine, in which form the model predicts them the best. The additional image manipulations we add to the images is **autocontrast** and **grayscale**. 

In [7]:
##############################
# here we collect all the paths
# to our train / test images
pathToImages = []
for root, dirs, files in os.walk("validation_set/", topdown=False): 
    for name in files:
        path = os.path.join(root, name)
        if path.endswith("jpg"):
            pathToImages.append(path) 

In [12]:
### COLLECT RAW IMAGES AND APPLY AUTOCONTRAST TO THEM BEFORE FEEDING THEM IN THE MODEL
autocontrastImages = []
labels = []
lettersDict = {'A' :0, 'B':1, 'U':2, 'V':3, 'E': 4} 


# Loops through imagepaths to load images and labels into arrays
for path in pathToImages:
    img = Image.open(path) 
    # applying autocontrast method  
    img = ImageOps.autocontrast(img, cutoff = 1, ignore = 1) 
    numpyimg = np.array(img)
    numpyimg = cv2.resize(numpyimg, (128, 128)) 
    #autocontrastImages.append(numpyimg)
    
    try:
        label = path.split("/")[1].split(".")[0][0]
        autocontrastImages.append(numpyimg)
    except Exception as e:
        print(e)
        print(path)
    letterToNumber = lettersDict.get(label)
    labels.append(letterToNumber)
    
autocontrastImages = np.array(autocontrastImages, dtype="uint8")
labels = np.array(labels)

In [14]:
######### COLLECT RAW IMAGES - NO EMPTY CLASS, 4 Handgestures
X = [] # Image data
y = [] # Labels

# Loops through imagepaths to load images and labels into arrays
for path in pathToImages:
    # Reads image and returns np.array
    img = cv2.imread(path) 
    img = cv2.resize(img, (128, 128)) 

    try:
        label = path.split("/")[1].split(".")[0][0]
        X.append(img)
    except Exception as e:
        print(e)
        print(path)
    letterToNumber = lettersDict.get(label)
    y.append(letterToNumber)
    
X = np.array(X, dtype="uint8")
y = np.array(y)

In [15]:
##model 5 RESNET50 with NO AUTOCONTRAST
## compared to IMAGE WITH NO autocontrast
## ADDED EMPTY CLASS 
## 5 CLASSES
model = load_model('./sign_language_dataset/model_res50_5classes_autocontrast.h5')
predictions = model.predict(X) # Make predictions towards the test set
#print(predictions)

probabilityThreshold = 0.7
cond = (predictions.max(axis=1) < probabilityThreshold).astype(float)
new_probability = np.hstack((predictions, cond[:,None]))
newy_pred = np.argmax(new_probability, axis=1) # Transform predictions into 1-D array with label number

pd.DataFrame(confusion_matrix(y, newy_pred), 
             columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4", "No Class"],
             index=["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4", "Actual No Class"])

,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4,No Class
Actual 0,79,2,0,22,2,21
Actual 1,0,81,1,0,0,15
Actual 2,0,0,16,10,0,39
Actual 3,5,1,0,46,7,9
Actual 4,0,0,0,0,19,0
Actual No Class,0,0,0,0,0,0


In [16]:
##model 5 RESNET50 with NO AUTOCONTRAST
## compared to IMAGE WITH autocontrast
## ADDED EMPTY CLASS 
## 5 CLASSES
model = load_model('./sign_language_dataset/model_res50_5classes_autocontrast.h5')
predictions = model.predict(autocontrastImages) # Make predictions towards the test set
#print(predictions)

probabilityThreshold = 0.7
cond = (predictions.max(axis=1) < probabilityThreshold).astype(float)
new_probability = np.hstack((predictions, cond[:,None]))
newy_pred = np.argmax(new_probability, axis=1) # Transform predictions into 1-D array with label number

pd.DataFrame(confusion_matrix(labels, newy_pred), 
             columns=["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4", "No Class"],
             index=["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4", "Actual No Class"])

,Predicted 0,Predicted 1,Predicted 2,Predicted 3,Predicted 4,No Class
Actual 0,82,0,3,22,5,14
Actual 1,3,73,0,0,5,16
Actual 2,0,0,43,11,1,10
Actual 3,5,2,5,42,5,9
Actual 4,0,0,0,0,19,0
Actual No Class,0,0,0,0,0,0
